# 🛠️ Titanic Survival Prediction - Data Assessment & Cleaning

This notebook is part of the Titanic Survival Prediction project. In this phase, we focus on Hyperparameter tuning the extra trees and randomforest model.

In [1]:
import numpy as np
import pandas as pd

from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

In [2]:
df = pd.read_csv("/kaggle/input/titanic-features/titanic_features.csv")
X, y = df.drop(columns=["Survived"]), df["Survived"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                   random_state=42)

In [3]:
!pip install mlflow==2.15.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.3/26.3 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 733.8/733.8 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 508.0/508.0 kB 21.0 MB/s eta 0:00:00
  Attempting uninstall: pytz
    Found existing installation: pytz 2025.2
    Uninstalling pytz-2025.2:
      Successfully uninstalled pytz-2025.2
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully unins

In [4]:
!pip install dagshub==0.3.34

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.6/236.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.4/238.4 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 3.8 MB/s eta 0:00:00
  Created wheel for fusepy: filename=fusepy-3.0.1-py3-none-any.whl size=10486 sha256=06ece4dffebb0923dc9b67e24e07c700df861b0068f96ec0272b0299355f660f
  Stored in directory: /root/.cache/pip/wheels/db/4a/86/fdda91f8b8ebb0a70e4181dc2423b1f70c3c2d3bd1158685b5
Successfully built fusepy
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.1.2
    Uninstalling tenacity-9.1.2:
      Successfully uninstalled tenacity-9.1.2
  Attempting uninstall: rich
    Found exi

In [5]:
import mlflow
import mlflow.sklearn
import dagshub
import os

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
dagshub_token = user_secrets.get_secret("DAGSHUB_PAT")

if not dagshub_token:
    raise EnvironmentError("DAGSHUB_PAT environment variable is not set")

os.environ["MLFLOW_TRACKING_USERNAME"] = dagshub_token
os.environ["MLFLOW_TRACKING_PASSWORD"] = dagshub_token

dagshub_url = "https://dagshub.com"
repo_owner = "pxxthik"
repo_name = "Titanic-Survival-Prediction"

# Set up MLflow tracking URI
mlflow.set_tracking_uri(f'{dagshub_url}/{repo_owner}/{repo_name}.mlflow')

In [6]:
mlflow.set_experiment("Hyperparameter tuning")

<Experiment: artifact_location='mlflow-artifacts:/c0edbb3929e2404686ce6d5bd63f4b8a', creation_time=1750269215534, experiment_id='3', last_update_time=1750269215534, lifecycle_stage='active', name='Hyperparameter tuning', tags={}>

In [7]:
# Hyperparameter grids
et_params = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 5]
}

rf_params = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 5]
}

tune_models = {
    "Random Forest": (RandomForestClassifier, rf_params),
    "Extra Trees": (ExtraTreesClassifier, et_params)
}

In [8]:
import uuid

for algo_name, (algo, param_grid) in tune_models.items():
    random_id = uuid.uuid4().hex[:4]
    parent_run_name = f"{algo_name}_HP_{random_id}"
    
    with mlflow.start_run(run_name=parent_run_name):
        # Perform grid search
        grid_search = GridSearchCV(algo(), param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        grid_search.fit(X_train, y_train)

        for params, mean_score, std_score in zip(grid_search.cv_results_['params'], grid_search.cv_results_['mean_test_score'], grid_search.cv_results_['std_test_score']):
            with mlflow.start_run(run_name=f"{algo_name} with params: {params}", nested=True):
                model = algo(**params)
                model.fit(X_train, y_train)

                # Model evaluation
                y_pred = model.predict(X_test)
                
                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred)
                recall = recall_score(y_test, y_pred)
                roc_auc = roc_auc_score(y_test, y_pred)

                # Log parameters and metrics
                mlflow.log_params(params)
                mlflow.log_metric("mean_cv_score", mean_score)
                mlflow.log_metric("std_cv_score", std_score)
                mlflow.log_metric("accuracy", accuracy)
                mlflow.log_metric("precision", precision)
                mlflow.log_metric("recall", recall)
                mlflow.log_metric("roc_auc", roc_auc)

                # Print the results for verification
                print(f"Mean CV Score: {mean_score}, Std CV Score: {std_score}")
                print(f"Accuracy: {accuracy}")
                print(f"Precision: {precision}")
                print(f"Recall: {recall}")
                print(f"roc_auc Score: {roc_auc}")
                

        # Log the best run details in the parent run
        best_params = grid_search.best_params_
        best_score = grid_search.best_score_
        mlflow.log_params(best_params)
        mlflow.log_metric("best_score", best_score)
        
        print(f"Best Params: {best_params}")
        print(f"Best Score: {best_score}")
    
        # Log model
        mlflow.sklearn.log_model(grid_search.best_estimator_, "model")

2025/06/18 18:16:52 INFO mlflow.tracking._tracking_service.client: 🏃 View run Random Forest with params: {'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 100} at: https://dagshub.com/pxxthik/Titanic-Survival-Prediction.mlflow/#/experiments/3/runs/97e801a99cd545f296380da19a6a6b8e.
2025/06/18 18:16:52 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/pxxthik/Titanic-Survival-Prediction.mlflow/#/experiments/3.


Mean CV Score: 0.8342361863488623, Std CV Score: 0.012670242613515672
Accuracy: 0.8044692737430168
Precision: 0.8
Recall: 0.7027027027027027
roc_auc Score: 0.7894465894465894
Mean CV Score: 0.8300009849305624, Std CV Score: 0.017923381988721558
Accuracy: 0.8156424581005587
Precision: 0.8153846153846154
Recall: 0.7162162162162162
roc_auc Score: 0.800965250965251


2025/06/18 18:16:58 INFO mlflow.tracking._tracking_service.client: 🏃 View run Random Forest with params: {'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 200} at: https://dagshub.com/pxxthik/Titanic-Survival-Prediction.mlflow/#/experiments/3/runs/53b708e1efbf4aa68e986bac2fc1e0c2.
2025/06/18 18:16:58 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/pxxthik/Titanic-Survival-Prediction.mlflow/#/experiments/3.


Mean CV Score: 0.8314291342460358, Std CV Score: 0.012184032953798225
Accuracy: 0.8044692737430168
Precision: 0.8
Recall: 0.7027027027027027
roc_auc Score: 0.7894465894465894


2025/06/18 18:17:08 INFO mlflow.tracking._tracking_service.client: 🏃 View run Random Forest with params: {'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 100} at: https://dagshub.com/pxxthik/Titanic-Survival-Prediction.mlflow/#/experiments/3/runs/49861cb0bdd9412095fc4998a8f7b3b9.
2025/06/18 18:17:08 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/pxxthik/Titanic-Survival-Prediction.mlflow/#/experiments/3.


...

2025/06/18 18:20:40 INFO mlflow.tracking._tracking_service.client: 🏃 View run Extra Trees with params: {'max_depth': None, 'min_samples_split': 5, 'n_estimators': 100} at: https://dagshub.com/pxxthik/Titanic-Survival-Prediction.mlflow/#/experiments/3/runs/a0fbb29b34d145669627008e73d3223e.
2025/06/18 18:20:40 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/pxxthik/Titanic-Survival-Prediction.mlflow/#/experiments/3.


Mean CV Score: 0.8019698611247907, Std CV Score: 0.005079740584514882
Accuracy: 0.8435754189944135
Precision: 0.8382352941176471
Recall: 0.7702702702702703
roc_auc Score: 0.8327541827541827


2025/06/18 18:20:50 INFO mlflow.tracking._tracking_service.client: 🏃 View run Extra Trees with params: {'max_depth': None, 'min_samples_split': 5, 'n_estimators': 200} at: https://dagshub.com/pxxthik/Titanic-Survival-Prediction.mlflow/#/experiments/3/runs/e499b101730c48048cf174ab7f84e5dc.
2025/06/18 18:20:50 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/pxxthik/Titanic-Survival-Prediction.mlflow/#/experiments/3.


Best Params: {'max_depth': 5, 'min_samples_split': 5, 'n_estimators': 200}
Best Score: 0.8229685807150595


/usr/local/lib/python3.11/dist-packages/_distutils_hack/__init__.py:16: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/_distutils_hack/__init__.py:31: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
2025/06/18 18:21:00 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the 